In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statistics import median,mean
%matplotlib inline
from pylab import rcParams
import sympy as sym

In [ ]:
#モーションキャプチャデータの読み込み(全身)
data_mocap = pd.read_csv("/Users/takeshin/ws/BerkeleyMHAD/Mocap/OpticalData/moc_s01_a01_r01.txt",sep=' ',header=None)

In [ ]:
#座標値列の列名作成する
p0 = ['x','y','z']
marker =[]
p2 =[]
for a in range(1,44):       #Mocapマーカー数:43個
    p1='m'+ str(a)
    for b in p0:
        p2 = p1 + b
        marker.append(p2)

In [ ]:
#座標値列の列名変更
data_mocap=data_mocap.rename(columns={data_mocap.columns[a]:marker[a] for a in range(129)} )
#座標値列以外の列名変更
data_mocap=data_mocap.rename(columns={129:'FrameNumber',130:'TimeStamp'})
#data_mocap.head()

In [ ]:
#mocapの左手首のデータを取得
mc_lw = data_mocap[['m16x','m16y','m16z','TimeStamp']]
#mc_lw = data_mocap[['m17x','m17y','m17z','TimeStamp']]
mc_lw.head()

In [ ]:
#fix NaN
for  c  in  ['m16x','m16y','m16z']:
    col = mc_lw[c]  #'X'列から1列ずつ見る
    for r in range(len(mc_lw)):  #1行ずつ見る
        if col[r] == 0.0:       #0値が見つかれば
            mc_lw.iloc[r][c] = 'NaN' #'NaN'を代入
mc_lw_clean = mc_lw.interpolate()

In [ ]:
#dawn sampling (avg)
#初期値を設定
i = 0
mc_lw_ds = pd.DataFrame()
#処理
while  i < (len(mc_lw_clean)//16)*16:
    mc_lw_ds= mc_lw_ds.append(mc_lw_clean.iloc[i : i+16,0:4].mean(),ignore_index=True)
    i+=16
    
mc_lw_ds= mc_lw_ds.append(mc_lw_clean.iloc[i : i+15,0:4].mean(),ignore_index=True)

In [ ]:
mc_lw_ds.head()

In [ ]:
def cal_va(data):
    diffs = pd.DataFrame()
    for i in range(len(data)-1):
        diff = data.iloc[i+1]-data.iloc[i]
        diffs = diffs.append(diff,ignore_index=True)
        
    vs = pd.DataFrame()
    for i in range(len(diffs)):
        v = diffs.iloc[i][1:]/diffs.iloc[i]["TimeStamp"]
        vs = vs.append(v,ignore_index=True)
    return vs

In [ ]:
#x座標 v[mm/s]
x_v = cal_va(mc_lw_ds).iloc[:,0]
#y座標 v[mm/s]
y_v = cal_va(mc_lw_ds).iloc[:,1]
#z座標 v[mm/s]
z_v = cal_va(mc_lw_ds).iloc[:,2]

In [ ]:
times = mc_lw_ds['TimeStamp'].shift(periods=-1)
mc_lw_v = pd.DataFrame({"xv":x_v,"yv":y_v,"zv":z_v,"TimeStamp":times})
mc_lw_v = mc_lw_v.dropna()
mc_lw_v.head()
mc_lw_v.shape

In [ ]:
mc_lw_v.head()

In [ ]:
#加速度計算 acceleration
#x
x_a = cal_va(mc_lw_v).iloc[:,0]
#y
y_a = cal_va(mc_lw_v).iloc[:,1]
#z
z_a = cal_va(mc_lw_v).iloc[:,2]

In [ ]:
times = mc_lw_v['TimeStamp'].shift(periods=-1)
mc_lw_a = pd.DataFrame({"xa":x_a,"ya":y_a,"za":z_a,"TimeStamp":times})
mc_lw_a = mc_lw_a.dropna()

In [ ]:
mc_lw_a.head()

In [ ]:
#単位を[G]に変換  [mm/s^2] →  [G]
accel = mc_lw_a[['xa','ya','za']]*0.001/9.80665
accel = accel.rolling(window= 10, center=False).mean()
accel = pd.concat([accel,mc_lw_a["TimeStamp"]], axis=1)

In [ ]:
accel = accel.dropna()
accel.head()


In [ ]:
mc_a_x = accel[['TimeStamp']]
mc_a_y1 = accel[['xa']]
mc_a_y2 = accel[['ya']]
mc_a_y3 = accel[['za']]

rcParams['figure.figsize'] = 16,10
plt.plot(mc_a_x,mc_a_y1,label = "x",color="red")
plt.plot(mc_a_x,mc_a_y2,label = "Motion Capture",color="blue")
plt.plot(mc_a_x,mc_a_y3,label = "z",color="green")
plt.legend()

plt.xlabel("time stamp [s]")
plt.ylabel("accel [G]")
plt.title("Acceleration")
plt.plot()

# ----------------------------

In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler
from scipy import stats as s
from sklearn.metrics import classification_report


In [3]:
def col_name():
    p0 = ['x','y','z']
    marker =[]
    p2 =[]
    for a in range(1,44):       #Mocapマーカー数:43個
        p1='m'+ str(a)
        for b in p0:
            p2 = p1 + b
            marker.append(p2)
    return marker

In [4]:
def read_mocap(file):
    data_mocap = pd.read_csv(file,sep=' ',header=None)
    #座標値列の列名変更
    data_mocap=data_mocap.rename(columns={data_mocap.columns[a]:col_name()[a] for a in range(129)} )
    #座標値列以外の列名変更
    data_mocap=data_mocap.rename(columns={129:'FrameNumber',130:'TimeStamp'})
    return data_mocap

In [5]:
#fix NaN
def fix_NaN(data):
    for  c  in  data.columns:
        col = data[c]  #'X'列から1列ずつ見る
        for r in range(len(data)):  #1行ずつ見る
            if col[r] == 0.0:       #0値が見つかれば
                data.iloc[r][c] = 'NaN' #'NaN'を代入
    mc_clean = data.interpolate()
    mc_clean = mc_clean.fillna(0)
    return mc_clean

def obtain_statistical_feature(window):
    #features = np.array(np.array(np.var(window, axis=0)))
    #features=np.array(np.array(np.mean(window-np.mean(window), axis=0)))
    features=np.array( np.array(np.mean(window-np.median(window), axis=0))) 
    features=np.append(features, np.array(np.std(window, axis=0))) 
    features=np.append(features, np.array(np.var(window, axis=0)))
    features=np.append(features, np.array(s.skew(window, axis=0)))
    features=np.append(features, np.array(s.kurtosis(window, axis=0)))
    features=np.append(features, np.array(get_tw_variance(window)))
    return features.reshape(1,len(features))

def get_tw_variance(window):
    total_cols = window.shape[1]
    tw_var = []
    for i in range (total_cols):
        column = window.iloc[:,i]
        tw_var.append(get_tw_col_var(column))
    return tw_var


def get_tw_col_var(column):
    mean = np.mean(column)
    total = len(column)
    
    v_sum = 0
    for i in range(total):
        w = np.exp(-0.5*(total-i))
        #print("el peso", w, "indice", i, "total", total)
        v = w*np.square(column.iloc[i]-mean)
        v_sum += v
    return v_sum

In [6]:
#速度加速度を求める関数を定義
def cal_va(data):
    diffs = pd.DataFrame()
    for i in range(len(data)-1):
        diff = data.iloc[i+1]-data.iloc[i]
        diffs = diffs.append(diff,ignore_index=True) 
    vs = pd.DataFrame()
    for i in range(len(diffs)):
        v = diffs.iloc[i][1:]/diffs.iloc[i]["TimeStamp"]
        vs = vs.append(v,ignore_index=True)
    return vs

In [7]:
def main_functuion(sampling_rate,*markers):
    x = np.empty([0,3*6*len(markers)])
    y = np.empty([0,])
    #Read all mocap data
    #subjects =  ["01","02","03","04","05","06","07","08","09","10","11","12"]
    #actions = ["01","02","03","04","05","06","07","08","09","10","11"]
    subjects =  ["01","02"]#,"03","04"]
    actions = ["01","02"]#,"03","04","05"]
    for subject in subjects:
        for action in actions:
            for record in range(1,3):
                try:
                    data = read_mocap('/Users/takeshin/ws/BerkeleyMHAD/Mocap/OpticalData/moc_s'+str(subject)+'_a'+str(action)+'_r0'+str(record)+'.txt')
                    print('moc_s'+str(subject)+'_a'+str(action)+'_r0'+str(record)+'.txt')
                except:
                    print("No File")
                #data = data[['m'+str(marker_n)+'x','m'+str(marker_n)+'y','m'+str(marker_n)+'z','TimeStamp']]
                columns = [['m'+str(i)+'x','m'+str(i)+'y','m'+str(i)+'z'] for i in markers]
                cols = np.array(columns)
                cols = list(np.ravel(cols))
                cols.append("TimeStamp")
                data = data[cols]
                data = fix_NaN(data)
                
                i = 0
                mc_ds = pd.DataFrame()
                #処理
                while  i < (len(data)//(480//sampling_rate))*(480//sampling_rate):
                    mc_ds = mc_ds.append(data.iloc[i : i+(480//sampling_rate),:].mean(),ignore_index=True)
                    i+=(480//sampling_rate)
                     
                mc_ds = mc_ds.append(data.iloc[i : i+(480//sampling_rate)-1,:].mean(),ignore_index=True)
                
                #x座標 v[mm/s]
                #x_v = cal_va(mc_ds).iloc[:,0]
                #y座標 v[mm/s]
                #y_v = cal_va(mc_ds).iloc[:,1]
                #z座標 v[mm/s]
                #z_v = cal_va(mc_ds).iloc[:,2]
                v = cal_va(mc_ds)
                times = mc_ds['TimeStamp'].shift(periods=-1)
                mc_v = pd.DataFrame(v)
                mc_v["TimeStamp"]=times
                mc_v = mc_v.dropna()
                
                
                #加速度計算 acceleration
                #x_a = cal_va(mc_v).iloc[:,0]
                #y
               # y_a = cal_va(mc_v).iloc[:,1]
                #z
                #z_a = cal_va(mc_v).iloc[:,2] 
                
                a = cal_va(mc_v)
                times = mc_v['TimeStamp'].shift(periods=-1)
                mc_a = pd.DataFrame(a)
                mc_a["TimeStamp"]=times
                mc_a = mc_a.dropna()
                
                 
                #単位を[G]に変換  [mm/s^2] →  [G]
                accel = mc_a.loc[:, mc_a.columns != 'TimeStamp']*0.001/9.80665
                accel = accel.rolling(window= 10, center=False).mean()
                accel = accel.dropna()
                sf = obtain_statistical_feature(accel)
                #accel = pd.concat([accel,mc_a["TimeStamp"]], axis=1)
                
                
                
                x =np.append(x,sf, axis=0)
                y = np.append(y,[action], axis=0)
            
    return x, y

In [8]:
x,y=main_functuion(30,16,17,18)   

moc_s01_a01_r01.txt
moc_s01_a01_r02.txt
moc_s01_a02_r01.txt
moc_s01_a02_r02.txt
moc_s02_a01_r01.txt
moc_s02_a01_r02.txt
moc_s02_a02_r01.txt
moc_s02_a02_r02.txt


In [ ]:
len(x[0])

In [ ]:
def ml_ar(X,y):                              
    #Try classifier
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('clf', SVC(kernel='linear')))
    model = Pipeline(estimators)
    
    seed = 11
    kf = KFold(n_splits=3, random_state=seed, shuffle=True)
    accuracy = [] 
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train)
        acc_train = model.score(X_train, y_train)
        acc_test = model.score(X_test, y_test)
        accuracy.append(acc_test)
        #print('Accuracy on training set: {:.2f}'
        #  .format(model.score(X_train, y_train)))
        #print('Accuracy on test set: {:.2f}'
        #  .format(model.score(X_test, y_test)))
        y_pred = model.predict(X_test)
        conf = confusion_matrix(y_test, y_pred)
        precision = precision_score(y_test, y_pred,average='macro') 
        recall = recall_score(y_test, y_pred,average='macro')
        print("*Accuracy* train:",acc_train, "test:",acc_test)
        #print(conf)
        #print ("precision:",precision,"recall:",recall)
        actions = ["01","02","03","04","05","06","07","08","09","10","11"]
        #actions = ["Jumping in place","Jumping jacks","Bending-hands up all the way down","Punching(boxing)","Waving-two hands","Waving - one hand (right)","Clapping hands","Throwing a ball","09","10","11"]
        print(classification_report(y_test, y_pred, target_names=actions))
        print("-----------")
    print("*test_ave*",sum(accuracy)/len(accuracy))

In [ ]:
ml_ar(x,y)

In [ ]:
def select_markers(*markers):
    #markers = [16,17]
    columns = [['m'+str(i)+'x','m'+str(i)+'y','m'+str(i)+'z'] for i in markers]
    #columns = reduce(add, columns)
    #columns.append("TimeStamp")
    cols = np.array(columns)
    cols = list(np.ravel(cols))
    cols.append("TimeStamp")
    return cols
    #return columns

In [ ]:
columns = [['m'+str(i)+'x','m'+str(i)+'y','m'+str(i)+'z'] for i in [16,17]]  
cols = np.array(columns)
cols = list(np.ravel(cols))
cols.append("TimeStamp")

In [ ]:
[cols]